In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('playlist_spotify.csv')

In [12]:
from dotenv import load_dotenv
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth


load_dotenv(".env")

scope = 'user-library-read,playlist-modify-public,playlist-modify-private'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=os.getenv('SPOTIPY_CLIENT_ID'),
    client_secret=os.getenv('SPOTIPY_CLIENT_SECRET'),
    redirect_uri=os.getenv('SPOTIPY_REDIRECT_URI'),
    scope=scope
))

In [13]:
n_clusters = df['cluster'].max() + 1
print(n_clusters)

5


In [14]:
import time
from tqdm import tqdm

def add_tracks_to_playlist(playlist_id, tracks):
    print(f"Añadiendo {len(tracks)} pistas a la playlist {playlist_id}")
    for i in tqdm(range(0, len(tracks), 100)):  # Dividir en lotes de 100
        fails = 0
        batch = tracks[i:i + 100]
        try:
            results = sp.playlist_add_items(playlist_id, batch)
        except spotipy.exceptions.SpotifyException as e:
            print(f"Error al añadir pistas a la playlist {playlist_id}: {e}")
            # Esperar medio segundo antes de intentar de nuevo
            time.sleep(0.5)
            fails += 1
            if fails > 10:
                print(f"Demasiados fallos, pasando al siguiente cluster")
                return
            continue



for i in tqdm(range(n_clusters), desc="Creando listas de reproducción", unit="cluster"):
    cluster_tracks = df[df['cluster'] == i]
    if not cluster_tracks.empty:
        playlist_name = f"Cluster {i+1} Playlist"
        
        try:
            # Crear la playlist
            playlist = sp.user_playlist_create(user=sp.current_user()['id'], name=playlist_name)
        except spotipy.exceptions.SpotifyException as e:
            print(f"Error al crear la playlist {playlist_name}: {e}")
            continue
        
        track_ids = cluster_tracks['id'].tolist()  # Obtener los IDs de las pistas directamente del DataFrame
        
        if track_ids:
            try:
                add_tracks_to_playlist(playlist['id'], track_ids)
            except spotipy.exceptions.SpotifyException as e:
                print(f"Error al añadir pistas a la playlist {playlist_name}: {e}")
        else:
            print(f"No se añadieron pistas a la playlist: {playlist_name}")

Creando listas de reproducción:   0%|          | 0/5 [00:00<?, ?cluster/s]

Añadiendo 35 pistas a la playlist 2joPxAmtfVgVcYglmKW6go


Creando listas de reproducción:  20%|██        | 1/5 [00:00<00:03,  1.01cluster/s]

Lista de reproducción creada: Cluster 1 Playlist
Añadiendo 516 pistas a la playlist 2mHvPN0YMbdmt58K4DLchF


Creando listas de reproducción:  40%|████      | 2/5 [00:03<00:05,  1.82s/cluster]

...

Lista de reproducción creada: Cluster 3 Playlist
Añadiendo 14 pistas a la playlist 2LSYlJno7JMFKKwCdHj857


Creando listas de reproducción:  80%|████████  | 4/5 [00:05<00:01,  1.15s/cluster]

Lista de reproducción creada: Cluster 4 Playlist
Añadiendo 5 pistas a la playlist 0jpZHTLbe4RF6ZZl9Q3C1a


Creando listas de reproducción: 100%|██████████| 5/5 [00:05<00:00,  1.18s/cluster]

Lista de reproducción creada: Cluster 5 Playlist
